In [1]:
!pip install -q transformers==4.57.1 peft datasets accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import torch


KeyboardInterrupt: 

In [ ]:
import os

In [ ]:
print(os.chdir("/content/drive/MyDrive"))

None


In [10]:
dataset = load_dataset("json", data_files={
    "train": "/content/drive/MyDrive/Mtech_Final_Project/train.json",
    "test": "/content/drive/MyDrive/Mtech_Final_Project/test.json"
})

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 94051
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 10451
    })
})


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
          (dense): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear8bitLt(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear8bitLt(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_

Tokenize the data

We’ll combine prompt + completion to train a causal model.

In [ ]:
# Add padding token
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def format_example(example):
    prompt = example["prompt"]
    completion = example["completion"]

    # Ensure prompt and completion are strings by joining if they are lists
    if isinstance(prompt, list):
        prompt = " ".join(prompt)
    if isinstance(completion, list):
        completion = " ".join(completion)

    return tokenizer(
        prompt + " " + completion,
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_datasets = dataset.map(format_example)

Map:   0%|          | 0/94051 [00:00<?, ? examples/s]

Map:   0%|          | 0/10451 [00:00<?, ? examples/s]

Configure LoRA

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # works well for transformer attention layers
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


Set up training configuration

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    eval_strategy="steps",  # Corrected argument name
    save_strategy="steps", # Corrected argument name
    eval_steps=200,
    save_steps=200,
    logging_steps=50,
    warmup_steps=50,
    fp16=True,
    report_to="none",
    load_best_model_at_end=True, # Load best model at the end of training
    disable_tqdm=False,              # Enable progress bar

)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

Save and push your model

In [ ]:
model.save_pretrained("./lora_model")
tokenizer.save_pretrained("./lora_model")


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./lora_model", tokenizer=tokenizer, device_map="auto")

prompt = """GNN summary: top_nodes=[14, 11]; scores=[0.97, 0.88]; implicated_lines=22, 23, 24; severity=HIGH; GNNExplainer_affecting:{}
System logs: Voltage imbalance detected in grid zone 4.
Task: Provide a concise diagnosis and recommended actions.

Diagnosis:"""

response = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)
print(response[0]['generated_text'])


In [ ]:
# Colab QLoRA training (optimized for T4 16GB / 4.5 hr sessions)
# Requirements (run once in a cell):
# !pip install -q transformers==4.57.1 datasets accelerate peft bitsandbytes

import os
import json
import math
import random
import time # Import time for sleep
from pathlib import Path

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# -----------------------
# USER CONFIG (change as needed)
# -----------------------
MODEL_NAME = "facebook/opt-1.3b"    # choose a model that works well on T4 in 4-bit (e.g., opt-1.3b, llamacpp-compatible small models)
TRAIN_FILE = "train.json"          # your train.json (json lines with {"prompt": "...", "completion": "..."} )
TEST_FILE  = "test.json"
OUTPUT_DIR = "/content/drive/MyDrive/Mtech_Final_Project/grid_llm_qora"  # set to a Google Drive path if you mounted drive (recommended)
USE_GOOGLE_DRIVE = True           # set True if you mounted drive
MAX_SAMPLES = None                # set to an int to quick-test (e.g., 2000). None uses all samples.
MAX_STEPS = 1500                  # stop early (useful for Colab). Set None to use full epochs.
EPOCHS = 3                        # will be ignored if MAX_STEPS is set
BATCH_SIZE = 1                    # per-device batch size; T4 may require 1
GRAD_ACC = 4                      # gradient_accumulation_steps
LEARNING_RATE = 2e-4
SAVE_STEPS = 200
EVAL_STEPS = 200
MAX_RETRIES = 5                   # Number of retries for dataset loading
RETRY_DELAY = 10                  # Delay in seconds between retries

# -----------------------
# Optional: mount Google Drive if using it
# -----------------------
if USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive_mount_path = "/content/drive"
    # Check if the mountpoint is accessible
    if not os.path.exists(drive_mount_path):
        print(f"Google Drive mountpoint {drive_mount_path} not found. Please ensure Google Drive is mounted.")
    else:
        drive.mount(drive_mount_path, force_remount=True)
        os.makedirs(OUTPUT_DIR, exist_ok=True)
else:
    os.makedirs(OUTPUT_DIR, exist_ok=True)


# Quick GPU check
print("CUDA available:", torch.cuda.is_available())
try:
    !nvidia-smi
except:
    pass

# -----------------------
# Load dataset with retry mechanism
# -----------------------
print("Loading dataset...")
# Check if the dataset files exist before loading
train_file_path = "/content/drive/MyDrive/Mtech_Final_Project/train.json"
test_file_path = "/content/drive/MyDrive/Mtech_Final_Project/test.json"

print(f"Checking for train file at: {train_file_path}")
print(f"Checking for test file at: {test_file_path}")

dataset = None
for i in range(MAX_RETRIES):
    if not os.path.exists(train_file_path) or not os.path.exists(test_file_path):
        print(f"Dataset files not found at {train_file_path} and {test_file_path}. Please ensure the files exist and the Google Drive is mounted correctly.")
        break # Exit retry loop if files are definitely not there
    else:
        print("Dataset files found. Attempting to load dataset...")
        try:
            dataset = load_dataset("json", data_files={"train": train_file_path, "test": test_file_path})
            print("Dataset loaded successfully.")
            break # Exit retry loop on success
        except OSError as e:
            print(f"Attempt {i+1}/{MAX_RETRIES} failed to load dataset: {e}")
            if i < MAX_RETRIES - 1:
                print(f"Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                print("Max retries reached. Could not load dataset.")

# optional quick subset to iterate fast (uncomment to debug)
if dataset is not None and MAX_SAMPLES is not None:
    dataset["train"] = dataset["train"].select(range(min(MAX_SAMPLES, len(dataset["train"]))))
    dataset["test"]  = dataset["test"].select(range(min(MAX_SAMPLES//10 if MAX_SAMPLES else len(dataset["test"]), len(dataset["test"]))))

if dataset is not None:
    print(dataset)

# -----------------------
# Prepare tokenizer
# -----------------------
if dataset is not None:
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # -----------------------
    # Quantization config for 4-bit QLoRA
    # -----------------------
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",                # nf4 recommended
        bnb_4bit_compute_dtype=torch.float16      # float16 for T4 (bfloat16 not supported well on T4)
    )

    # -----------------------
    # Load model in 4-bit
    # -----------------------
    print("Loading model in 4-bit (this may take a while)...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto"   # allow HF to place layers across devices
    )

    # Prepare model for k-bit training (freezes norms etc)
    model = prepare_model_for_kbit_training(model)

    # Enable gradient checkpointing to save memory
    model.gradient_checkpointing_enable()

    # -----------------------
    # Apply LoRA (PEFT)
    # -----------------------
    print("Applying LoRA adapters...")
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],  # focus on Q and V projection matrices
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # -----------------------
    # Tokenize / format examples
    # -----------------------
    max_length = 512
    def preprocess(example):
        # Combine prompt + completion together for causal LM training.
        # We will set labels = input_ids and mask padding with -100 (so loss ignores padding)
        text = (example["prompt"].strip() + " " + example["completion"].strip())
        enc = tokenizer(text,
                        truncation=True,
                        max_length=max_length,
                        padding="max_length")
        input_ids = enc["input_ids"]
        attention_mask = enc["attention_mask"]
        labels = [((tok if tok != tokenizer.pad_token_id else -100)) for tok in input_ids]
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

    print("Tokenizing dataset (may take a few minutes)...")
    tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

    # convert to torch format is done by Trainer automatically if columns exist as lists

    # -----------------------
    # Data collator
    # -----------------------
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # -----------------------
    # Training arguments (Colab optimized)
    # -----------------------
    if MAX_STEPS is not None:
        max_steps_val = MAX_STEPS
        num_train_epochs = EPOCHS  # still set but Trainer will stop at max_steps
    else:
        max_steps_val = -1
        num_train_epochs = EPOCHS

    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACC,
        logging_steps=50,
        evaluation_strategy="steps",
        eval_steps=EVAL_STEPS,
        save_strategy="steps",
        save_steps=SAVE_STEPS,
        save_total_limit=3,
        learning_rate=LEARNING_RATE,
        fp16=torch.cuda.is_available(),
        max_steps=max_steps_val if max_steps_val > 0 else None,
        num_train_epochs=num_train_epochs,
        remove_unused_columns=False,
        push_to_hub=False,
        load_best_model_at_end=False,
    )

    # -----------------------
    # Trainer
    # -----------------------
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # -----------------------
    # Train (with try/except for safe checkpointing)
    # -----------------------
    print("Starting training...")
    try:
        trainer.train(resume_from_checkpoint=False)
    except Exception as e:
        print("Training interrupted with exception:", e)
        print("Saving model checkpoint to OUTPUT_DIR so you can resume later.")
        trainer.save_model(OUTPUT_DIR)

    # -----------------------
    # Save final model (LoRA adapters + quantized base)
    # -----------------------
    print("Saving final model...")
    model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print("Model saved to:", OUTPUT_DIR)

    # -----------------------
    # Quick test: generate on a few prompts
    # -----------------------
    from transformers import pipeline
    gen_pipe = pipeline("text-generation", model=OUTPUT_DIR, tokenizer=tokenizer, device_map="auto")
    sample_prompt = dataset["test"][0]["prompt"]
    print("\nSample prompt:\n", sample_prompt)
    out = gen_pipe(sample_prompt, max_new_tokens=120, do_sample=True, temperature=0.7)
    print("\nModel output:\n", out[0]["generated_text"])

Mounted at /content/drive
CUDA available: True
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Thu Oct 16 07:27:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             31W /   70W |    9898MiB /  15360MiB | 

# **Actual Code**

In [1]:
# Requirements (run once in a cell):
!pip install -q transformers==4.57.1 datasets accelerate peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 11.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
# Imports
import os
import json
import math
import random
import time # Import time for sleep
from pathlib import Path

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [9]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/drive/MyDrive/Mtech_Final_Project/llm_dataset.jsonl")
train_test = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_test["train"].to_json("/content/drive/MyDrive/Mtech_Final_Project/train.json")
train_test["test"].to_json("/content/drive/MyDrive/Mtech_Final_Project/test.json")

print(train_test)

Creating json from Arrow format:   0%|          | 0/95 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 94051
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 10451
    })
})


In [7]:
# -----------------------
# USER CONFIG (change as needed)
# -----------------------
MODEL_NAME = "facebook/opt-1.3b"    # choose a model that works well on T4 in 4-bit (e.g., opt-1.3b, llamacpp-compatible small models)
TRAIN_FILE = "train.json"          # your train.json (json lines with {"prompt": "...", "completion": "..."} )
TEST_FILE  = "test.json"
OUTPUT_DIR = "/content/drive/MyDrive/Mtech_Final_Project/grid_llm_qora"  # set to a Google Drive path if you mounted drive (recommended)
USE_GOOGLE_DRIVE = True           # set True if you mounted drive
MAX_SAMPLES = None                # set to an int to quick-test (e.g., 2000). None uses all samples.
MAX_STEPS = 1500                  # stop early (useful for Colab). Set None to use full epochs.
EPOCHS = 3                        # will be ignored if MAX_STEPS is set
BATCH_SIZE = 1                    # per-device batch size; T4 may require 1
GRAD_ACC = 4                      # gradient_accumulation_steps
LEARNING_RATE = 2e-4
SAVE_STEPS = 200
EVAL_STEPS = 200
MAX_RETRIES = 5                   # Number of retries for dataset loading
RETRY_DELAY = 10                  # Delay in seconds between retries

In [10]:
# -----------------------
# Optional: mount Google Drive if using it
# -----------------------
if USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive_mount_path = "/content/drive"
    # Check if the mountpoint is accessible
    if not os.path.exists(drive_mount_path):
        print(f"Google Drive mountpoint {drive_mount_path} not found. Please ensure Google Drive is mounted.")
    else:
        drive.mount(drive_mount_path, force_remount=True)
        os.makedirs(OUTPUT_DIR, exist_ok=True)
else:
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Quick GPU check
print("CUDA available:", torch.cuda.is_available())
try:
    !nvidia-smi
except:
    pass

Mounted at /content/drive
CUDA available: False
/bin/bash: line 1: nvidia-smi: command not found


In [7]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [11]:
dataset = load_dataset("json", data_files={
    "train": "/content/drive/MyDrive/Mtech_Final_Project/train.json",
    "test": "/content/drive/MyDrive/Mtech_Final_Project/test.json"
})

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 94051
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 10451
    })
})


In [12]:
# optional quick subset to iterate fast (uncomment to debug)
if dataset is not None and MAX_SAMPLES is not None:
    dataset["train"] = dataset["train"].select(range(min(MAX_SAMPLES, len(dataset["train"]))))
    dataset["test"]  = dataset["test"].select(range(min(MAX_SAMPLES//10 if MAX_SAMPLES else len(dataset["test"]), len(dataset["test"]))))

if dataset is not None:
    print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 94051
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 10451
    })
})


In [13]:
# -----------------------
# Prepare tokenizer
# -----------------------
if dataset is not None:
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # -----------------------
    # Quantization config for 4-bit QLoRA
    # -----------------------
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",                # nf4 recommended
        bnb_4bit_compute_dtype=torch.float16      # float16 for T4 (bfloat16 not supported well on T4)
    )

    # -----------------------
    # Load model in 4-bit
    # -----------------------
    print("Loading model in 4-bit (this may take a while)...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto"   # allow HF to place layers across devices
    )

    # Prepare model for k-bit training (freezes norms etc)
    model = prepare_model_for_kbit_training(model)

    # Enable gradient checkpointing to save memory
    model.gradient_checkpointing_enable()

    # -----------------------
    # Apply LoRA (PEFT)
    # -----------------------
    print("Applying LoRA adapters...")
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],  # focus on Q and V projection matrices
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Loading model in 4-bit (this may take a while)...


pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Applying LoRA adapters...
trainable params: 1,572,864 || all params: 1,317,330,944 || trainable%: 0.1194


In [25]:
# -----------------------
# Tokenize / format examples
# -----------------------
# Tokenize function
def tokenize_fn(examples):
    inputs = tokenizer(
        examples["prompt"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    labels = tokenizer(
        examples["completion"],
        max_length=256,
        truncation=True,
        padding="max_length",
    )
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply to both splits in batched mode
tokenized = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["prompt", "completion"]
)

    # convert to torch format is done by Trainer automatically if columns exist as lists

Map:   0%|          | 0/94051 [00:00<?, ? examples/s]

Map:   0%|          | 0/10451 [00:00<?, ? examples/s]

In [26]:
# Verify correct shapes
print(tokenized)
print(tokenized["train"][0].keys())
print(len(tokenized["train"][0]["input_ids"]))  # should be ~512

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 94051
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10451
    })
})
dict_keys(['input_ids', 'attention_mask', 'labels'])
512


In [27]:
batch = tokenized["train"][0]
for k, v in batch.items():
    print(k, type(v), len(v) if hasattr(v, '__len__') else v)

input_ids <class 'list'> 512
attention_mask <class 'list'> 512
labels <class 'list'> 256


In [28]:
# -----------------------
# Data collator
# -----------------------
if dataset is not None:
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [29]:
# -----------------------
# Training arguments (Colab optimized)
# -----------------------
if dataset is not None:
    if MAX_STEPS is not None:
        max_steps_val = MAX_STEPS
        num_train_epochs = EPOCHS  # still set but Trainer will stop at max_steps
    else:
        max_steps_val = -1
        num_train_epochs = EPOCHS

    training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,     #  smaller batch
    gradient_accumulation_steps=1,     #  fewer accumulations
    learning_rate=5e-5,
    num_train_epochs=1,                #  1 epoch max for CPU
    max_steps=200,                     #  stop early (optional)
    logging_steps=10,
    eval_strategy="no",          #  disable evaluation for faster training
    save_strategy="no",                #  don’t save checkpoints
    fp16=False,                        #  must be off for CPU
    bf16=False,                        #  off
    remove_unused_columns=False,
    push_to_hub=False,
)


In [30]:
# -----------------------
# Trainer
# -----------------------
if dataset is not None:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

/tmp/ipython-input-3201098384.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# -----------------------
# Train (with try/except for safe checkpointing)
# -----------------------
if dataset is not None:
    print("Starting training...")
    try:
        trainer.train(resume_from_checkpoint=False)
    except Exception as e:
        print("Training interrupted with exception:", e)
        print("Saving model checkpoint to OUTPUT_DIR so you can resume later.")
        trainer.save_model(OUTPUT_DIR)

Starting training...


In [21]:
# -----------------------
# Save final model (LoRA adapters + quantized base)
# -----------------------
if dataset is not None:
    print("Saving final model...")
    model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print("Model saved to:", OUTPUT_DIR)

Saving final model...
Model saved to: /content/drive/MyDrive/Mtech_Final_Project/grid_llm_qora


In [22]:
# -----------------------
# Quick test: generate on a few prompts
# -----------------------
if dataset is not None:
    from transformers import pipeline
    gen_pipe = pipeline("text-generation", model=OUTPUT_DIR, tokenizer=tokenizer, device_map="auto")
    sample_prompt = dataset["test"][0]["prompt"]
    print("\nSample prompt:\n", sample_prompt)
    out = gen_pipe(sample_prompt, max_new_tokens=120, do_sample=True, temperature=0.7)
    print("\nModel output:\n", out[0]["generated_text"])

Device set to use cpu



Sample prompt:
 GNN summary: top_nodes=[22, 17, 20]; scores=[1.0, 0.743, 0.743]; implicated_lines=48, 52, 58, 67, 69; severity=HIGH; GNNExplainer_affecting:{"22": [8, 9, 10, 11, 12, 18, 19, 22], "17": [14, 15, 16, 17, 20, 21], "20": [14, 15, 16, 17, 20, 21, 23]}
System logs: <SYSTEM_LOGS_HERE>
Task: Consider you are and electrical engineer and you get the summary of the grid, your task is to explain the scenario to the worker so that he can detect that actual fault in grid

Diagnosis:

Model output:
 GNN summary: top_nodes=[22, 17, 20]; scores=[1.0, 0.743, 0.743]; implicated_lines=48, 52, 58, 67, 69; severity=HIGH; GNNExplainer_affecting:{"22": [8, 9, 10, 11, 12, 18, 19, 22], "17": [14, 15, 16, 17, 20, 21], "20": [14, 15, 16, 17, 20, 21, 23]}
System logs: <SYSTEM_LOGS_HERE>
Task: Consider you are and electrical engineer and you get the summary of the grid, your task is to explain the scenario to the worker so that he can detect that actual fault in grid

Diagnosis:
Diagnosis requires 